In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
dataset = pd.read_csv('../input/sentiment140/training.1600000.processed.noemoticon.csv', encoding='latin1', header=None)
dataset.head()

In [ ]:
dataset.isna().sum()
print (dataset.values[0:1])
dataset[0].unique()

In [ ]:
x = dataset[5]
y = dataset[0].values

In [ ]:
import re

hashtags = re.compile(r"^#\S+|\s#\S+")
mentions = re.compile(r"^@\S+|\s@\S+")
urls = re.compile(r"https?://\S+")

def process_text(text):
  text = hashtags.sub('hashtag', text)
  text = mentions.sub('entity', text)
  return text.strip().lower()
  
def match_expr(pattern, string):
  return not pattern.search(string) == None

def get_data_wo_urls(dataset):
    link_with_urls = dataset.text.apply(lambda x: match_expr(urls, x))
    return dataset[[not e for e in link_with_urls]]

x=x.apply(process_text)

In [ ]:
print (x[0])

In [ ]:
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

x=x.apply(remove_urls)
print (x[0])

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_numeric, strip_punctuation, strip_short, stem_text

In [ ]:
x=x.apply(remove_stopwords)
x=x.apply(strip_numeric)
x=x.apply(strip_punctuation)
x=x.apply(strip_short)
x=x.apply(stem_text)

In [ ]:
print(x.values[0:5])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, random_state=0, test_size=0.25)

In [ ]:
print(x_train[0])

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, SimpleRNN, Embedding, Dropout, Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)

list_tokenized_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

maxlen = 130
x_train = pad_sequences(list_tokenized_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

embed_size = 32
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add((SimpleRNN(32, return_sequences=False)))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 64
epochs = 5
hist=model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))